In [1]:
load('sidon_cryptosystem.sage')

In [2]:
##Implementation of the first minrank attack in sidon cryptosystem paper
##We first define the sidon space we are trying to break
q = 7
basefield = GF(q)
k = 3
r = 2
rk = r*k
y, F, F_r, d, c = ConstructSidon2k(q, k)
##Construct the attacker's F_r
y2, F2, F_r2, d2, c2 = ConstructSidon2k(q, k)
matrixList, sidonbasis, mult_table, F_r_basis , origbasis = publicKey(y,q,F,F_r)

[2 6 0 6 2 4]
[5 1 4 0 3 3]
[0 2 0 4 1 4]
[2 6 1 2 0 1]
[6 3 6 3 6 3]
[5 6 2 3 4 4]


In [88]:
##Construct the attacker's basis of F and F_r
iterations = 0
sidonbasis = Matrix(basefield, k, lambda i,j: basefield.random_element()*0)
##basis of F
while sidonbasis.is_invertible() == False and iterations < 500:
    element = F.random_element()
    sidonbasis = Matrix([list(vector(element^(q^i))) for i in range(k)])
    iterations += 1
v = [F2(list(sidonbasis[i])) for i in range(k)]
sidonbasis = [j*y2*y2^-1 for  j in v]
sidonbasismat = Matrix([convertToLong(i) for i in sidonbasis])
##Extend F to F_r
F_r2basis = Matrix(basefield, k, lambda i,j: basefield.random_element()*0)
while F_r2basis.is_invertible() == False and iterations < 500:
    F_r2basis = sidonbasismat
    stackmat = Matrix(basefield, rk - k, rk, lambda i, j: basefield.random_element())
    F_r2basis = F_r2basis.stack(stackmat)
    iterations += 1
    print(F_r2basis.rank(), F_r2basis.nrows(), F_r2basis.ncols())

6 6 6


In [89]:
F_r2basisels =  [convertFromLong(F_r2basis[i], F2, F_r2) for i in range(rk)]

In [90]:
##define variables
g = ['g' + str(i) for i in range(rk)]

In [130]:
##create a polynomial ring over the variabels in order to do symbolic calculations 
b = [["b" + str(j) + str(i) for i in range(rk)] for j in range(rk)]
u = [["u" + str(j) + str(i) for i in range(k) for j in range(k)]]
bvec = []
uvec = []
for i in b: 
    bvec += i
for i in u: 
    uvec += i
R = PolynomialRing(F_r2, g + bvec + uvec)
indeterminates = R.gens()
R.inject_variables()

for i in range(rk): 
    g[i] = R(g[i])
for i in range(k*k): 
    uvec[i] = R(uvec[i])
for i in range(rk*rk):
    bvec[i] = R(bvec[i])
y2 = vector(g).row()*vector(F_r2basisels).column()
F_r2_q = vector([R(i^q) for i in F_r2basisels])

Defining g0, g1, g2, g3, g4, g5, b00, b01, b02, b03, b04, b05, b10, b11, b12, b13, b14, b15, b20, b21, b22, b23, b24, b25, b30, b31, b32, b33, b34, b35, b40, b41, b42, b43, b44, b45, b50, b51, b52, b53, b54, b55, u00, u10, u20, u01, u11, u21, u02, u12, u22


In [131]:
v = [] 
for i in range(k):
    v += vector(uvec[i*k:(i+1)*k]).row()*vector(F_r2basisels[0:k]).column() + y2*vector(uvec[i*k:(i+1)*k]).row()*vector(F_r2_q[0:k]).column() 

In [144]:
B = [0 for i in range(rk)]
for i in range(rk): 

    for j in range(rk):
        B[i] += R(bvec[i*rk + j])*R(F_r2_q[j])

In [133]:
bvec[0:rk]

[b00, b01, b02, b03, b04, b05]

In [134]:
v_mult_table =[[v[i]*v[j] for i in range(len(v))] for j in range(len(v))]

In [171]:
B_table = [[0 for i in range(k)] for j in range(k)]
eq_table = [[0 for i in range(k)] for j in range(k)]
for i in range(k): 
    for j in range(k): 
        sum1 = 0
        for l in range(rk): 
            sum1 += R(matrixList[l][i][j])*B[l]
        B_table[i][j] = sum1  
        eq_table[i][j] = R(v_mult_table[i][j])  - sum1

In [170]:
(convertToLong(F_r2(list(eq_table[0][0])[0][0]))*F_r2basis.inverse()*g0).parent()

Multivariate Polynomial Ring in g0, g1, g2, g3, g4, g5, b00, b01, b02, b03, b04, b05, b10, b11, b12, b13, b14, b15, b20, b21, b22, b23, b24, b25, b30, b31, b32, b33, b34, b35, b40, b41, b42, b43, b44, b45, b50, b51, b52, b53, b54, b55, u00, u10, u20, u01, u11, u21, u02, u12, u22 over Univariate Quotient Polynomial Ring in a over Finite Field in z of size 7^3 with modulus a^2 + (z^2 + 3*z + 6)*a + 2*z^2 + 3*z + 2

In [179]:
vectorized_eq_table = [[0 for i in range(k)] for j in range(k)]
for i in range(k): 
    for j in range(k): 
        if i > j: 
            continue
        else: 
            eqvec = list(eq_table[i][j])
            sumvec = vector([0 for l in range(rk)])
            for term in eqvec: 
                coeff = term[0]
                monomial = term[1]
                coeff = convertToLong(F_r2(coeff))
                rep = coeff*F_r2basis.inverse()
                sumvec += rep*monomial
        vectorized_eq_table[i][j] = sumvec

In [192]:
list(vectorized_eq_table[0][1][0])

[(4, g0^2*u00*u01),
 (1, g0*g1*u00*u01),
 (3, g1^2*u00*u01),
 (6, g1*g2*u00*u01),
 (6, g0*g3*u00*u01),
 (1, g1*g3*u00*u01),
 (1, g2*g3*u00*u01),
 (4, g3^2*u00*u01),
 (2, g0*g4*u00*u01),
 (3, g1*g4*u00*u01),
 (5, g2*g4*u00*u01),
 (6, g3*g4*u00*u01),
 (2, g0*g5*u00*u01),
 (3, g1*g5*u00*u01),
 (5, g2*g5*u00*u01),
 (5, g3*g5*u00*u01),
 (1, g4*g5*u00*u01),
 (1, g5^2*u00*u01),
 (3, g0^2*u10*u01),
 (3, g1^2*u10*u01),
 (3, g2^2*u10*u01),
 (1, g1*g3*u10*u01),
 (5, g3^2*u10*u01),
 (5, g1*g4*u10*u01),
 (5, g2*g4*u10*u01),
 (5, g3*g4*u10*u01),
 (1, g4^2*u10*u01),
 (5, g1*g5*u10*u01),
 (5, g2*g5*u10*u01),
 (1, g3*g5*u10*u01),
 (4, g4*g5*u10*u01),
 (1, g0^2*u20*u01),
 (1, g0*g1*u20*u01),
 (4, g1^2*u20*u01),
 (6, g0*g2*u20*u01),
 (6, g0*g3*u20*u01),
 (6, g1*g3*u20*u01),
 (4, g3^2*u20*u01),
 (2, g1*g4*u20*u01),
 (2, g3*g4*u20*u01),
 (1, g4^2*u20*u01),
 (2, g1*g5*u20*u01),
 (1, g3*g5*u20*u01),
 (5, g4*g5*u20*u01),
 (3, g5^2*u20*u01),
 (3, g0^2*u00*u11),
 (3, g1^2*u00*u11),
 (3, g2^2*u00*u11),
 (1, g1*g

In [149]:
##Algebraic attack implementation is also here
def algebraicAttack(q,k,a,b):
    basefield = GF(q)
    rk = 2*k
    y, F, F_r, d, c = ConstructSidon2k(q, k)
    ##Construct the attacker's F_r
    y2, F2, F_r2, d2, c2 = ConstructSidon2k(q, k)
    matrixList, sidonbasis, mult_table, F_r_basis , origbasis = publicKey(y,q,F,F_r)
    rhs = [a.row()*i*b.column() for i in matrixList]
    R_ = PolynomialRing(GF(q), ['x' + str(i) for i in range(2*k)] + ['xn'], order = "lex")
    indeterminates = R_.gens()[:-1]
    xn = R_.gens()[-1]
    R_.inject_variables()
    a1 = vector(indeterminates[0:k])
    b1 = vector(indeterminates[k:])
    system = [a1.row()*i*b1.column() for i in matrixList]
    system = [system[i][0][0] - rhs[i][0][0]*xn^2 for i in range(len(system))]
    I = ideal(system)
    gb = I.groebner_basis()
    I = ideal(gb)
    I  = I.subs(x0 = 1)
    I = I.subs(xn= 1)
    R_  = R_.remove_var(xn)
    R_  = R_.remove_var(x0)
    gens = [R_(i) for i in I.gens()]
    I = ideal(gens)
    I = I.groebner_basis()
    I = I.ideal()
    return I.variety(), matrixList

In [156]:
q = 7
k = 4
a = vector([5,1,2,1])
b = vector([1,2,1,4])
c, d = algebraicAttack(q,k,a,b)

[4 5 1 4 6 5 1 1]
[3 4 3 2 5 3 0 2]
[2 6 3 5 1 4 6 1]
[1 0 0 6 4 1 6 0]
[3 1 3 6 4 1 0 5]
[2 0 1 6 2 1 6 0]
[3 0 6 4 0 4 3 6]
[4 0 1 1 2 3 1 4]
Defining x0, x1, x2, x3, x4, x5, x6, x7, xn


In [52]:
a1 = vector(indeterminates[0:k])
b1 = vector(indeterminates[k:])
system = [a1.row()*i*b1.column() for i in matrixList]

In [151]:
c

[{x5: 5, x4: 3, x3: 5, x2: 6, x1: 3}, {x5: 2, x4: 1, x3: 5, x2: 1, x1: 2}]

[[6], [4], [4], [6], [3], [3]]
[[6], [4], [4], [6], [3], [3]]


In [67]:
R_.is_commutative()

True

In [59]:
system[0][0][0].parent()

TypeError: x must have length self.ngens()

x

In [114]:
I  = I.subs(xn= 1, x0 = 1)

KeyError: 'xn'

In [58]:
I.dimension()

2

In [59]:
I = I.groebner_basis()

In [60]:
I = ideal(I)

In [85]:
v = I.subs(x0 = 5, x1 = 1, x2 = 2)
v = v.subs(x3 = 2,x4  = 1, x5 = 1)

In [87]:
v.variety()

ValueError: The dimension of the ideal is 7, but it should be 0

In [86]:
v.dimension()

7

In [64]:
I.dimension()

2

In [88]:
v

Ideal (0, 0, 0, 0, 0) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Finite Field of size 7

In [89]:
I.remove_variable(xn)

AttributeError: 'MPolynomialIdeal' object has no attribute 'remove_variable'

In [93]:
R_  = R_.remove_var(xn)

In [97]:
gens = [R_(i) for i in I.gens()]

In [104]:
v = ideal(gens)

In [105]:
v.dimension()

0

In [101]:
v = v.subs(x0 = 5)

In [102]:
R_  = R_.remove_var(x0)

In [103]:
gens = [R_(i) for i in v.gens()]

In [106]:
v.variety()

[{x5: 1, x4: 1, x3: 2, x2: 2, x1: 1}, {x5: 5, x4: 6, x3: 2, x2: 6, x1: 6}]